<a href="https://colab.research.google.com/github/18ME10049/18ME10049/blob/main/Project_on_NLP_18ME10049.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# Project  on Natural Language Processing

## Duration : 15th Nov - 10 Dec, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this Project is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



# Word Based LSTM model

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [4]:
# Load the data and preprocess data and store corpus in raw_text
# Load the data and preprocess data and store corpus in raw_text
with open('corpus (2).txt','r',encoding="utf8") as file:
    data = file.read()
file.close()

raw_text = data.lower()

In [41]:
import string
def  clean_my_corpos(doc):

    doc = doc.replace('”', '"')
    doc = doc.replace('“', '"')
    doc = doc.replace('_',"")
    doc = doc.replace(';','.')
    doc = doc.replace('—','')
    return doc

In [141]:
my_text = clean_my_corpos(raw_text)


In [45]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [48]:
my_text

'chapter i.\ndown the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, "and what is the use of a book," thought alice "without pictures or conversations?" so she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a white rabbit with pink eyes ran close by her.\nthere was nothing so very remarkable in that. nor did alice think it so very much out of the way to hear the rabbit say to itself, "oh dear!\noh dear!\ni shall be late!" (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural). but when the rabbit actually took a watch out of its waist

In [57]:
# tokenize sentences
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts([my_text])
word_index = tokenizer.word_index

tokens = tokenizer.texts_to_sequences([my_text])[0]

In [71]:
print("Size of Word_index is : ", 1+len(word_index))
print(tokens)
print("Total number of words :",len(tokens))
print("Total Unique words : ",len(set(tokens)))
vocab_size =  1+len(word_index)

Size of Word_index is :  2764
[305, 10, 36, 2, 101, 695, 11, 14, 279, 4, 104, 27, 475, 8, 349, 75, 17, 423, 19, 2, 1065, 3, 8, 350, 131, 4, 52, 137, 56, 601, 6, 22, 834, 63, 2, 326, 17, 423, 14, 835, 23, 7, 22, 43, 696, 56, 1066, 12, 7, 3, 29, 35, 2, 212, 8, 5, 326, 58, 11, 173, 696, 56, 1066, 25, 6, 14, 836, 12, 17, 351, 327, 16, 67, 16, 6, 55, 24, 2, 476, 150, 146, 17, 424, 27, 602, 3, 540, 328, 2, 1067, 8, 425, 5, 1485, 1486, 48, 26, 697, 2, 541, 8, 190, 38, 3, 1068, 2, 1487, 54, 280, 5, 147, 101, 20, 1488, 158, 230, 281, 75, 17, 57, 14, 131, 25, 27, 1069, 12, 15, 837, 66, 11, 86, 7, 25, 27, 89, 34, 8, 2, 80, 4, 253, 2, 101, 92, 4, 254, 125, 151, 125, 151, 10, 174, 26, 542, 54, 6, 58, 7, 112, 1070, 7, 1071, 4, 17, 15, 6, 255, 4, 53, 1489, 18, 30, 23, 18, 2, 60, 7, 21, 163, 81, 698, 23, 54, 2, 101, 1490, 179, 5, 426, 34, 8, 76, 1072, 477, 3, 105, 18, 7, 3, 42, 329, 19, 11, 1073, 4, 17, 205, 24, 7, 1491, 603, 17, 327, 15, 6, 22, 103, 127, 256, 5, 101, 20, 352, 5, 1072, 477, 56, 5, 426

In [72]:
dataX = []
dataY = []
seq_length = 50
for i in range(0, len(tokens)-seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is <OOV>
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)

N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  26519


In [ ]:
print(dataX[0],dataY[0])

[314, 15, 36, 2, 95, 729, 11, 13, 288, 5, 106, 27, 493, 8, 368, 72, 18, 444, 20, 2, 1131, 4, 8, 369, 142, 5, 58, 143, 60, 730, 7, 22, 876, 63, 2, 346, 18, 444, 13, 877, 31, 9, 22, 54, 731, 60, 1132, 12, 9, 77] 37


In [73]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [77]:
X

array([[ 305,   10,   36, ...,   12,    7,    3],
       [  10,   36,    2, ...,    7,    3,   29],
       [  36,    2,  101, ...,    3,   29,   35],
       ...,
       [ 365,    3,  170, ...,    3,    2, 2763],
       [   3,  170,   88, ...,    2, 2763, 1464],
       [ 170,   88,  158, ..., 2763, 1464,  802]])

In [78]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [79]:
#with embedding
embedding_dim = 50
#seq_length = 50
# Deep Neural network with one embedding layer , one Bidirection LSTM , one Dense layer with softmax activation 

# model = keras.Sequential([
#     keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
#     keras.layers.Bidirectional(keras.layers.LSTM(64 , activation=tahh , use_bias=True , bias_initializer='zeros' , )),
#     keras.layers.Dense(vocab_size, activation='softmax')
# ])

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

# model compilation with crossentropy loss and adam optimization using metrics accuracy.
# compile model
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

# model summary
# model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            138200    
_________________________________________________________________
lstm_4 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2764)              279164    
Total params: 568,264
Trainable params: 568,264
Non-trainable params: 0
_________________________________________________________________
None


In [92]:
#Model 2

model2 = Sequential()
model2.add(Embedding(vocab_size, 100, input_length=seq_length))
model2.add(LSTM(100, return_sequences=True))
model2.add(LSTM(100))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(vocab_size, activation='softmax'))
print(model2.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 100)           276400    
_________________________________________________________________
lstm_6 (LSTM)                (None, 50, 100)           80400     
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 2764)              279164    
Total params: 726,464
Trainable params: 726,464
Non-trainable params: 0
_________________________________________________________________
None


In [80]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model

In [93]:
# compile model2
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model

In [ ]:
model.fit(X, y, batch_size=128, epochs=50)

In [ ]:
model2.fit(X, y, batch_size=128, epochs=100)

In [96]:
model.save('model.h5')
model.save('my_model_for_text_gen.h5')

In [114]:
from keras import models 
model1_with_50_epoch = models.load_model('model.h5')
History_model2_with_100_epoch = models.load_model('my_model_for_text_gen.h5') # Accuracy >= 50%

In [98]:
reverse_word_map = tokenizer.index_word ## Create word to idx map using tokenizer.word_index

In [137]:
def generate_seq(model, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = np.argmax(model.predict(encoded),axis=-1)
        #yhat = np.argmax(model.predict(encoded),axis=-1)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [143]:
# pick a random seed

start_index = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start_index]  #Random_sentence
input_str = ' '.join([reverse_word_map[value] for value in pattern])
real_answer = []
for i in range(start_index+50 , start_index + 50 + 50):
    real_answer.append(tokens[i])

real_sequance = ' '.join([reverse_word_map[value] for value in real_answer])

print("\nGiven Seed or Input: \n", input_str)
print("\nReal Answer according to corpos will be: \n", real_sequance)

answer1 = generate_seq(model1_with_50_epoch, seq_length, input_str, 100)
answer2 = generate_seq(History_model2_with_100_epoch, seq_length, input_str, 100)

print("\nPredict Sequence using model1: \n",answer1)
print("\nPredict Sequence using model2: \n",answer2)


Given Seed or Input: 
 chapter ix the mock turtle’s story you can’t think how glad i am to see you again you dear old thing said the duchess as she tucked her arm affectionately into alice’s and they walked off together alice was very glad to find her in such a pleasant temper and

Real Answer according to corpos will be: 
 thought to herself that perhaps it was only the pepper that had made her so savage when they met in the kitchen when i’m a duchess she said to herself not in a very hopeful tone though i won’t have any pepper in my kitchen at all soup does very

Predict Sequence using model1: 
 she was a little timidly said alice the gryphon was gone to me and the rabbit was in a little hot speaker soooop with the air the queen was very likely said alice indignantly and then the mock turtle went on and began whistling so she had been wandering much she had been to do it was the best butter i suppose i should think it was a little timidly said alice the gryphon was gone to me and sh

In [144]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
print(generate_seq(History_model2_with_100_epoch,seq_length, input_str , 10))

little sister of the house and the gryphon said alice


# Character based LSTM Model 1

In [ ]:
# User the preprocess data and create raw_text

# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))
tokenizer2 = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tokenizer2.fit_on_texts(raw_text)

char_to_int =  tokenizer2.word_index

In [ ]:
# Print the total characters and character vacob size
n_chars = len(raw_text)
n_vocab = len(char_to_int)
print("n_chars : ",n_chars)
print("n_vocab : ",n_vocab)

n_chars :  142477
n_vocab :  46


In [ ]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
tokens2 = tokenizer2.texts_to_sequences([raw_text])[0]
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = tokens2[i:i+seq_length]
    seq_out = tokens2[i + seq_length]
    
    if seq_out == 1:
        continue
    
    dataX.append(seq_in)
    dataY.append(seq_out)


n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  142377


In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.array(dataX)

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [ ]:
embedding_dim =100
max_length =100

In [ ]:
n_vocab = n_vocab+1
model = Sequential()
model.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 100, 100)          4700      
_________________________________________________________________
lstm_12 (LSTM)               (None, 256)               365568    
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 47)                12079     
Total params: 382,347
Trainable params: 382,347
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#implement mapping of integer to character
int_to_char = tokenizer2.index_word

In [ ]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	print("Given Sequence: \n", pattern)
	sequence = np.array(char_tokenizer.texts_to_sequences(pattern)[:100]).reshape(1,100)
	for i in range(x):
		prediction = model.predict(sequence, verbose=0)
		sequence = np.append(sequence,prediction.argmax())
		sequence = sequence[1:]
		sequence.resize(1,100) 
		final_string = ' '.join(pattern.split()[:100]) +  ''.join(char_tokenizer.sequences_to_texts(sequence))
	return final_string

In [ ]:
# pick a random seed
start_index = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start_index]
input_str = ''.join([int_to_char[value] for value in pattern])

answer1 = predict_next_100_chars(input_str,200)
print("Predict Sequence: \n",answer1)

Given Sequence: 
 ing anything more till the pigeon had finished.
“as if it wasn’t trouble enough hatching the eggs,” 
Predict Sequence: 
 ing anything more till the pigeon had finished. “as if it wasn’t trouble enough hatching the eggs,”  j u s t   a s   w e l l   s a y   i n   a   d e a r   o f   t h i n g   i   h a v e   n o   n o t i c e   t o   b e   a   p r o c e s s i o n , ”   t h o u g h t   a l i c e ,   “ i   d o n ’ t   k


In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.

 
str2 = predict_next_100_chars(input_str,200)
print("Predict Sequence: \n",str2)

Given Sequence: 
 The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not  a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue  him or his sheep.
Predict Sequence: 
 The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue him or his sheep.u c h e s s ,   “ a n d   t h e   m o r a l   o f   t h a t   i s — ‘ t h e   m o r e   t h a n   t h e   r e a s o n   i s   t h e   m o u s e   d i d   y o u   t o   d o   t o   d o   t h a t   i  


## Character based LSTM Model 2


In [ ]:
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2225/2225 [==============================] - 50s 22ms/step - loss: 2.1236
Epoch 2/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.5672
Epoch 3/20
2225/2225 [==============================] - 50s 22ms/step - loss: 1.3843
Epoch 4/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.2818
Epoch 5/20
2225/2225 [==============================] - 50s 22ms/step - loss: 1.2139
Epoch 6/20
2225/2225 [==============================] - 50s 22ms/step - loss: 1.1612
Epoch 7/20
2225/2225 [==============================] - 50s 22ms/step - loss: 1.1142
Epoch 8/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.0775
Epoch 9/20
2225/2225 [==============================] - 49s 22ms/step - loss: 1.0430
Epoch 10/20
2225/2225 [==============================] - 50s 22ms/step - loss: 1.0152
Epoch 11/20
2225/2225 [==============================] - 49s 22ms/step - loss: 0.9869
Epoch 12/20
2225/2225 [==============================] - 49s 22

In [ ]:
# Generate the sequence similar to above methods

'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	print("Given Sequence: \n", pattern)
	sequence = np.array(char_tokenizer.texts_to_sequences(pattern)[:100]).reshape(1,100)
	for i in range(x):
		prediction = model1.predict(sequence, verbose=0)
		sequence = np.append(sequence,prediction.argmax())
		sequence = sequence[1:]
		sequence.resize(1,100)
		final_string = ' '.join(pattern.split()[:100]) +  ''.join(char_tokenizer.sequences_to_texts(sequence))
	return final_string

In [ ]:
# pick a random seed
start_index = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start_index]
input_str = ''.join([int_to_char[value] for value in pattern])

answer2 = predict_next_100_chars(input_str,200)
print("Predict Sequence: \n",answer2)

Given Sequence: 
 rply; “i advise you to leave off this minute.” she generally gave herself very good advice, (though 
Predict Sequence: 
 rply; “i advise you to leave off this minute.” she generally gave herself very good advice, (thoughg   t o   b e   s u r p r i s e d   t o   s e e   t h a t   s h e   w a s   n o w   a b o u t   i t ,   a n d   t h e   w h o l e   p a r t y   w a s   t h e   w h i t e   r a b b i t   w a s   s i t


In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.

str1 = predict_next_100_chars(input_str,200)
print("Predict Sequence: \n",str1)

Given Sequence: 
 The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not  a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue  him or his sheep.
Predict Sequence: 
 The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue him or his sheep.e   t h a t   s h e   w a s   n o w   a b o u t   i t ,   a n d   t h e   w h o l e   p a r t y   w a s   t h e   w h i t e   r a b b i t   w a s   s i t t i n g   o n   t h e   d o o r ,   s h e   w


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?



```
# This is formatted as code
```

**Answer:** Word based LSTM model performs better on seen data as we can see in epoch 20 accuracy score is better then val_accuracy but also it is performing well in unseen data as well. And also models can approximate to the past data pretty well. But for the new data it is not perfect as on train data.

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:**  The Observation of output:
LSTM model2 is performing way better than model1. As in the model 1 same words are repeating and model2 sentence makes more sense as model2 is more complex and adds one extra unit than model1

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 9.0MB/s 
     |████████████████████████████████| 890kB 24.7MB/s 
     |████████████████████████████████| 2.9MB 41.9MB/s 
     |████████████████████████████████| 1.1MB 47.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5939d6a428a50ea6320ce8d1ecb47986e7745d3fd27a13f2551968541d68473b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [ ]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [ ]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)

Predict next  3  words


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 4, 30522])
['p', 'vol', 'math']
p    vol    math
